In [ ]:
"""
Training with 3 epochs
with yolo-pose-ano 3
on yolov8l pose

"""

In [ ]:
!pip install -q ultralytics opencv-python scikit-image
!pip install -q -U ipywidgets

In [ ]:
# import os
# import shutil
# from sklearn.model_selection import train_test_split

# # Base directory containing the video folders
# base_dir = '/kaggle/input/yolo-pose-ano-3/cricshot-yolo-ano'
# base_dir = '/kaggle/input/cric-pose-v2/cric_pose_v2'

# output_dir = '/kaggle/working/cricshot-split'

# # Create output directories for train, val, and test sets
# splits = ['train', 'val', 'test']
# for split in splits:
#     os.makedirs(os.path.join(output_dir, split), exist_ok=True)

# # Define the split ratios
# train_ratio = 0.7
# val_ratio = 0.2
# test_ratio = 0.1

# # Function to split files into train, val, and test
# def split_data(class_dir, files, train_ratio, val_ratio):
#     train_files, temp_files = train_test_split(files, test_size=(1 - train_ratio))
#     val_files, test_files = train_test_split(temp_files, test_size=(test_ratio / (val_ratio + test_ratio)))
#     return train_files, val_files, test_files

# # Traverse through each class folder
# for class_name in os.listdir(base_dir):
#     class_path = os.path.join(base_dir, class_name)
#     if os.path.isdir(class_path):
#         files = [f for f in os.listdir(class_path) if f.endswith('.mp4')]
        
#         # Split the files
#         train_files, val_files, test_files = split_data(class_path, files, train_ratio, val_ratio)

#         # Copy files to corresponding folders
#         for split, split_files in zip(splits, [train_files, val_files, test_files]):
#             split_class_dir = os.path.join(output_dir, split, class_name)
#             os.makedirs(split_class_dir, exist_ok=True)
#             for file in split_files:
#                 src = os.path.join(class_path, file)
#                 dst = os.path.join(split_class_dir, file)
#                 shutil.copyfile(src, dst)

# print("Dataset successfully split into train, val, and test sets.")

In [ ]:
# import os
# import cv2
# import numpy as np
# from skimage import io
# from tqdm import tqdm

# # Define paths
# dataset_dir = output_dir
# processed_data_dir = "processed_data"
# os.makedirs(processed_data_dir, exist_ok=True)

# # Function to extract and save frames from videos
# def process_videos(directory, output_dir, num_frames=32, img_size=(640, 640)):
#     categories = os.listdir(directory)
    
#     for category in categories:
#         category_path = os.path.join(directory, category)
#         output_category_path = os.path.join(output_dir, category)
#         os.makedirs(output_category_path, exist_ok=True)
        
#         video_files = os.listdir(category_path)
#         for video_file in tqdm(video_files, desc=f"Processing videos in {category}", leave=True):
#             video_path = os.path.join(category_path, video_file)
#             cap = cv2.VideoCapture(video_path)
#             frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#             interval = max(1, frame_count // num_frames)  # Ensure interval is at least 1

#             for i in range(num_frames):
#                 cap.set(cv2.CAP_PROP_POS_FRAMES, i * interval)
#                 ret, frame = cap.read()
#                 if not ret:
#                     break
#                 resized_frame = cv2.resize(frame, img_size)
#                 frame_filename = os.path.join(output_category_path, f"{os.path.splitext(video_file)[0]}_frame_{i}.jpg")
#                 cv2.imwrite(frame_filename, resized_frame)
#             cap.release()

# # Process train, val, and optionally test sets
# process_videos(os.path.join(dataset_dir, "train"), os.path.join(processed_data_dir, "train"))
# process_videos(os.path.join(dataset_dir, "val"), os.path.join(processed_data_dir, "val"))
# process_videos(os.path.join(dataset_dir, "test"), os.path.join(processed_data_dir, "test"))

In [ ]:
!cp -r /kaggle/input/trainable-yolo-pose-3/* /kaggle/working/

In [ ]:
data_yaml_content = """
path: /kaggle/working/processed_data  
train: train  
val: val     
test: test
nc: 10     
names:
  0: 'sweep'
  1: 'square_cut'
  2: 'hook'
  3: 'lofted'
  4: 'cover'
  5: 'late_cut'
  6: 'pull'
  7: 'defense'
  8: 'flick'
  9: 'straight'

"""


# Path where you want to save the data.yaml file
yaml_file_path = '/kaggle/working/data.yaml'

# Save the YAML content to the file
with open(yaml_file_path, 'w') as file:
    file.write(data_yaml_content)

# Confirm the file has been saved
print(f"data.yaml saved at: {yaml_file_path}")

In [ ]:
# Read the YAML file content (optional)
with open(yaml_file_path, 'r') as file:
    content = file.read()
    print(content)

In [ ]:
# Initialize Weights & Biases (W&B) in disabled mode.

import wandb
wandb.init(mode="disabled")

In [ ]:
from ultralytics import YOLO

data_file = '/kaggle/working/processed_data/'
# data_file = yaml_file_path

# Load and train the YOLOv8 model using the data.yaml file
model = YOLO('/kaggle/input/ano-3rd-49-eps/pytorch/default/1/best NA 49.pt')
# model = YOLO('yolov8l-cls.pt') 

model.train(
    data=data_file, 
    epochs=15, 
    imgsz=640
)

In [ ]:
import pandas as pd
import os

# Base directory for the test videos
test_base_dir = '/kaggle/working/cricshot-frames/test'  # Update this path to where your test videos are stored
test_base_dir = 'cricshot-split/test'

# Prepare the ground truth DataFrame by scanning the test directory
ground_truth_data = []
for class_name in os.listdir(test_base_dir):
    class_dir = os.path.join(test_base_dir, class_name)
    if os.path.isdir(class_dir):
        for video_name in os.listdir(class_dir):
            if video_name.endswith('.mp4'):  # Assuming the videos are in .mp4 format
                ground_truth_data.append({
                    'video_name': video_name,
                    'true_class': class_name
                })

# Convert to a DataFrame
ground_truth_df = pd.DataFrame(ground_truth_data)
print(ground_truth_df)

In [ ]:
import cv2
import os
import pandas as pd
from ultralytics import YOLO
from collections import Counter, defaultdict

saved_model_path = '/kaggle/input/v2-yolov8l-cls/pytorch/default/1/best (9).pt'
saved_model_path = '/kaggle/working/runs/classify/train/weights/best.pt'

# Load the trained YOLOv8 model
model = YOLO(saved_model_path)  # Update to your trained model path

# Initialize a list to store predictions
predictions = []

# Function to extract exactly 32 frames from a video
def extract_frames(video_path, num_frames=32, img_size=(640, 640)):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if frame_count < num_frames:
        raise ValueError(f"Video {video_path} has fewer frames ({frame_count}) than requested ({num_frames}).")
    
    interval = max(1, frame_count // num_frames)
    frames = []
    for i in range(num_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * interval)
        ret, frame = cap.read()
        if not ret:
            break
        resized_frame = cv2.resize(frame, img_size)
        frames.append(resized_frame)
    cap.release()
    return frames

# Loop through the test directories and videos
for class_name in os.listdir(test_base_dir):
    class_dir = os.path.join(test_base_dir, class_name)
    if os.path.isdir(class_dir):
        for video_name in os.listdir(class_dir):
            if video_name.endswith('.mp4'):
                video_path = os.path.join(class_dir, video_name)
                
                try:
                    # Extract 32 frames from the video
                    frames = extract_frames(video_path, num_frames=32)
                    
                    # Predict on video frames
                    results = model.predict(frames)
#                     print(results[0].probs.top1)
                    # Initialize a dictionary to accumulate class probabilities
                    class_probabilities = defaultdict(list)

                    # Collect frame-level probabilities
                    for result in results:
                        top_class = result.names[result.probs.top1]
                        top_prob = result.probs.top1conf.item()
                        class_probabilities[top_class].append(top_prob)
                    
                    # Calculate average probability for each class
                    avg_class_probabilities = {cls: sum(probs) / len(results) for cls, probs in class_probabilities.items()}
                    
                    print(avg_class_probabilities)
                    
                    # Determine the class with the highest average probability
                    most_probable_class = max(avg_class_probabilities, key=avg_class_probabilities.get)
                    
                    print(f"Video: {video_name}, Most Probable Class: {most_probable_class}, Avg Probability: {avg_class_probabilities[most_probable_class]:.4f}")
                    
                    # Store the prediction
                    predictions.append({
                        'video_name': video_name,
                        'true_class': class_name,
                        'predicted_class': most_probable_class,
                        'avg_probability': avg_class_probabilities[most_probable_class]
                    })
                except ValueError as e:
                    print(f"Skipping {video_path}: {e}")
                    
                    
# Convert predictions to a DataFrame
predictions_df = pd.DataFrame(predictions)
print(predictions_df)

# Example: Print the predicted class for the first video
if not predictions_df.empty:
    first_prediction = predictions_df.iloc[0]
    print(f"Video: {first_prediction['video_name']}, Predicted Class: {first_prediction['predicted_class']}")

In [ ]:
# Merge predictions with ground truth for comparison
comparison_df = predictions_df

# comparison_df.to_csv("comperision_df.csv",index=False)
# Display the comparison DataFrame
comparison_df

In [ ]:
import os
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

# Assume comparison_df is already created

# Calculate accuracy
accuracy = accuracy_score(comparison_df['true_class'], comparison_df['predicted_class'])
accuracy_percentage = accuracy * 100
accuracy_formatted = f"{accuracy:.6f}"
accuracy_percentage_formatted = f"{accuracy_percentage:.4f}%"
print(f"Accuracy: {accuracy_formatted} ({accuracy_percentage_formatted})")

# Generate classification report
class_report = classification_report(comparison_df['true_class'], comparison_df['predicted_class'], target_names=os.listdir(test_base_dir), digits=4)
print("Classification Report:\n", class_report)

# Generate confusion matrix
cm = confusion_matrix(comparison_df['true_class'], comparison_df['predicted_class'], labels=os.listdir(test_base_dir))

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=os.listdir(test_base_dir), yticklabels=os.listdir(test_base_dir))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.savefig('confusion_matrix.png')  # Save as PNG
plt.show()
plt.close()  # Close the plot to free memory

# Calculate classification errors
comparison_df['error'] = comparison_df['true_class'] != comparison_df['predicted_class']
error_count = comparison_df['error'].sum()
total_count = len(comparison_df)
error_rate = error_count / total_count
error_rate_percentage = error_rate * 100
error_rate_formatted = f"{error_rate:.4f}"
error_rate_percentage_formatted = f"{error_rate_percentage:.2f}%"

print(f"Total classification errors: {error_count}")
print(f"Error rate: {error_rate_formatted} ({error_rate_percentage_formatted})")

# Export the results
# 1. Save accuracy to a text file
with open('accuracy.txt', 'w') as f:
    f.write(f"Accuracy: {accuracy_formatted} ({accuracy_percentage_formatted})\n")

# 2. Save classification report to a text file
with open('classification_report.txt', 'w') as f:
    f.write("Classification Report:\n")
    f.write(class_report)

# 3. Save the comparison DataFrame including errors to a CSV file
comparison_df.to_csv('classification_errors.csv', index=False)

# 4. Save error summary to a text file
with open('error_summary.txt', 'w') as f:
    f.write(f"Total classification errors: {error_count}\n")
    f.write(f"Error rate: {error_rate_formatted} ({error_rate_percentage_formatted})\n")
